## Exploratory Data Analysis

In [3]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
#Import the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from scipy import stats
from sklearn.preprocessing import StandardScaler


In [16]:
# Doing some QOL improvements
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
#Import the dataset
df = pd.read_csv('github_issues_tickets.csv')

/tmp/ipykernel_77269/4151716196.py:2: DtypeWarning: Columns (43,48,49,52,53,104) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('github_issues_tickets.csv')


In [19]:
df = pd.read_csv("github_issues_tickets.csv", low_memory=False)
# df.iloc[:, [43, 48, 49, 52, 53, 104]].isna().sum()
# df.iloc[:, [43, 48, 49, 52, 53, 104]]

In [21]:
print(df.columns.tolist())

['answers_0_author', 'answers_0_body', 'answers_0_creation_time', 'answers_1_author', 'answers_1_body', 'answers_1_creation_time', 'answers_2_author', 'answers_2_body', 'answers_2_creation_time', 'answers_3_author', 'answers_3_body', 'answers_3_creation_time', 'answers_4_author', 'answers_4_body', 'answers_4_creation_time', 'answers_5_author', 'answers_5_body', 'answers_5_creation_time', 'answers_6_author', 'answers_6_body', 'answers_6_creation_time', 'answers_7_author', 'answers_7_body', 'answers_7_creation_time', 'answers_8_author', 'answers_8_body', 'answers_8_creation_time', 'answers_9_author', 'answers_9_body', 'answers_9_creation_time', 'assignee', 'assignee_id', 'assignee_login', 'assignee_type', 'assignee_url', 'body', 'closed_at', 'comments', 'comments_url', 'created_at', 'html_url', 'id', 'labels_0_color', 'labels_0_default', 'labels_0_description', 'labels_0_id', 'labels_0_name', 'labels_0_url', 'labels_10_color', 'labels_10_default', 'labels_10_description', 'labels_10_id',

In [22]:
#Checking for missing values
missing = df.isnull().mean().sort_values(ascending=False)
missing[missing > 0]  # only show columns with missing data


labels_10_description      1.000000
milestone                  1.000000
assignee                   1.000000
labels_10_name             0.999749
labels_10_color            0.999749
labels_10_url              0.999749
labels_10_default          0.999749
labels_10_id               0.999749
labels_9_description       0.999499
labels_9_name              0.998809
labels_9_color             0.998809
labels_9_url               0.998809
labels_9_id                0.998809
labels_9_default           0.998809
labels_8_description       0.998496
labels_8_url               0.997242
labels_8_id                0.997242
labels_8_default           0.997242
labels_8_color             0.997242
labels_8_name              0.997242
labels_7_description       0.995863
labels_7_url               0.994735
labels_7_default           0.994735
labels_7_name              0.994735
labels_7_id                0.994735
labels_7_color             0.994735
labels_6_description       0.993043
labels_6_name              0

Seems like there are a lot of missing values for columns. Will need to check if there is relevant data needed for edge cases or significant data, but it is unlikely. I will proceed to prune features that have more than **70% missingness**.

In [30]:
column_of_interest = 'answers_2_body'

# Show only rows where this column is not null
df[df[column_of_interest].notna()][column_of_interest][0]

'Following the history and looking at the page, it looks like this issue can be closed 😄 \r\n![Screenshot 2023-05-16 at 22 51 28](https://github.com/angular/angular/assets/11311806/3b6c1e3d-672c-4ef3-9f94-877c0f0f1a77)\r\n'

In [32]:
df.iloc[0]

answers_0_author                                                  MukulKolpe
answers_0_body                   Hey @alan-agius4, can I work on this issue?
answers_0_creation_time                            2023-05-05T18:09:31+00:00
answers_1_author                                                   JeanMeche
answers_1_body             @MukulKolpe You can open a PR if you'd like to...
answers_1_creation_time                            2023-05-05T18:19:32+00:00
answers_2_author                                           jamesanderson9182
answers_2_body             Following the history and looking at the page,...
answers_2_creation_time                            2023-05-16T21:51:45+00:00
answers_3_author                             angular-automatic-lock-bot[bot]
answers_3_body             This issue has been automatically locked due t...
answers_3_creation_time                            2023-06-19T00:09:04+00:00
answers_4_author                                                         NaN

In [33]:
import pandas as pd

def extract_answers(df):
    answer_rows = []
    
    for idx, row in df.iterrows():
        issue_id = row.get('id', idx)  # Use index if 'id' is missing
        
        for i in range(10):  # Assuming answers_0 to answers_9
            author = row.get(f'answers_{i}_author')
            body = row.get(f'answers_{i}_body')
            creation_time = row.get(f'answers_{i}_creation_time')

            # Skip empty answers
            if pd.notna(author) or pd.notna(body) or pd.notna(creation_time):
                answer_rows.append({
                    'issue_id': issue_id,
                    'answer_id': i,
                    'author': author,
                    'body': body,
                    'creation_time': creation_time
                })
    
    # Create a new long-format DataFrame
    answers_df = pd.DataFrame(answer_rows)
    return answers_df


In [35]:
df_long = extract_answers(df)
# Display the first few rows of the new DataFrame
df_long.head(10)

,issue_id,answer_id,author,body,creation_time
0,1697841153,0,MukulKolpe,"Hey @alan-agius4, can I work on this issue?",2023-05-05T18:09:31+00:00
1,1697841153,1,JeanMeche,@MukulKolpe You can open a PR if you'd like to...,2023-05-05T18:19:32+00:00
2,1697841153,2,jamesanderson9182,"Following the history and looking at the page,...",2023-05-16T21:51:45+00:00
3,1697841153,3,angular-automatic-lock-bot[bot],This issue has been automatically locked due t...,2023-06-19T00:09:04+00:00
4,2341142530,0,github-actions[bot],Hi I'm an AI powered bot that finds similar is...,2024-06-07T20:52:50+00:00
5,2341142530,1,codendone,"This is very similar to #9670, though that iss...",2024-06-08T02:22:56+00:00
6,2341142530,2,codendone,This was closed as fixed/duplicate of #7629. T...,2024-07-11T02:02:16+00:00
7,343146499,0,Neme12,Why is this a bug?,2018-09-24T20:37:57+00:00
8,343146499,1,dpoeschl,Design Meeting Notes:\r\n\r\nSince `s` is `rea...,2018-09-24T22:49:18+00:00
9,343146499,2,dpoeschl,@Neme12 is there a good use case for this that...,2018-09-24T22:49:22+00:00


In [44]:
# Check NaN values in the new DataFrame

df_long.isna().sum()

issue_id         0
answer_id        0
author           0
body             0
creation_time    0
dtype: int64

In [48]:
# Check for missing values
print(df_long.isna().sum())

# Sample the data
print(df_long.sample(5))

# Basic stats
print(df_long.describe(include='all'))

# Count how many answers each issue has
print(df_long["issue_id"].value_counts())


issue_id         0
answer_id        0
author           0
body             0
creation_time    0
dtype: int64
         issue_id  answer_id                                author  \
20547   483569471          5  microsoft-github-policy-service[bot]   
48002   822668602          6                        clairernovotny   
29924   137217026          1                               flip111   
11594  1266031193          4                               pablobm   
8228    152998601          2                               maarekj   

                                                    body  \
20547  This issue has not had any activity in 6 month...   
48002  @mjfara I would recommend using a custom deleg...   
29924  It would help to know which class `$entity->ge...   
11594  Fixed by https://github.com/thoughtbot/adminis...   
8228   I fix the errors.\nThe main error was related ...   

                   creation_time  
20547  2023-11-16T05:23:15+00:00  
48002  2021-03-27T19:38:09+00:00  
29924

In [84]:
df_thread = df_long.sort_values(by=["issue_id", "creation_time"])

Upon further discovery, I realized that there are bot data included in this dataset which might not be relevant to the LLM. I will proceed to remove data from these bots.

In [85]:
df_no_bots = df_thread[~df_thread["author"].str.contains(r"\[bot\]$", na=False)]

In [86]:
df_no_bots.head(10)

,issue_id,answer_id,author,body,creation_time
53074,456983,0,almet,"If we do add the SITEURL value, it can break b...",2010-12-05T17:31:55+00:00
53075,456983,1,Gui13,"A bit hackish for sure, but I'm not yet good e...",2010-12-05T17:43:43+00:00
53076,456983,2,almet,"okay, let's go for that so. Will do that when ...",2010-12-05T17:45:28+00:00
53077,456983,3,almet,"arnaud is currently working on that, see his f...",2010-12-14T15:57:04+00:00
53078,456983,4,Gui13,Should this bug be closed now?\n,2010-12-21T21:02:19+00:00
53079,456983,5,Gui13,Well... clicked the wrong button. It is now.\n,2010-12-21T21:02:53+00:00
53080,456983,6,almet,not yet. Arnaud is still working on that (and ...,2010-12-22T02:13:48+00:00
44636,648603,0,codesplicer,Having a similar issue when rendering partials...,2011-03-05T18:40:27+00:00
44637,648603,1,indutny,"Problem is in partials itself, you're not addi...",2011-03-14T10:48:22+00:00
44638,648603,2,tj,the partials directory lookup has been removed...,2011-03-14T14:14:40+00:00


## Data Cleaning

In [98]:
import re

def clean_text(text):
    if pd.isna(text):
        return ""
    # Keep code snippets (don't remove backticks or code blocks)
    text = re.sub(r"http\S+", "<link>", text)  # Replace URLs
    text = re.sub(r"@\w+", "", text)           # Remove @mentions
    text = re.sub(r"#\d+", "", text)           # Remove issue references like #123
    text = re.sub(r"\s+", " ", text)           # Collapse whitespace
    return text.strip()


df_clean = df_no_bots.copy()
df_clean["clean_body"] = df_no_bots["body"].apply(clean_text)



In [99]:
df_clean[df_clean["clean_body"].str.contains("http")]["clean_body"].iloc[0]


'Relay doesn\'t support "merging" queries together. Instead, we recommend creating a custom network layer that takes the list of queries passed to `sendQueries` and sends them to the server in a single http request. This is what we do in our internal network layer implementation. I wrote a bit more about how to do this [here](<link> but you can use the getID() method on each query request as a key to identify which data goes with which query.'

In [100]:
# Save cleaned DataFrame to CSV
df_clean.to_csv("cleaned_helpdesk_data.csv", index=False, encoding="utf-8")
